Summary:
- parse, beautiful soup, pass only if codes are 200
- get all texts, if there was more time, searching for divs to avoid pages with multiple products would be better
- pass some samples through gpt parser, some pages contain title that can be used to check if parser output is ok, input could be vectorised and picked by similarity to some templates, but is just cut, as usually most important info is up.
  prompt extracts product name, type, color, and other attributes, (I would change prompt to return specific type of furniture, as categories returned are too broad)
- retrain llama2 7b on the data. When quantized isn't significally slower than newest encoder, or encoder-decoder architecture models(that are better for abstractive summary generation, bert would be good for extractive. This just a test, but I guess in real life abstractive would be more interesting for parsing company data) There was not enough data to train the llama properly, and I didn't want to overfit it.
- some analysis in bokeh
- chroma vector store with llama, very simple. Without langchain.

The code isn't as clean as I'd like, but I had no time to clean it better and add the comments.

In [200]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [258]:
import pandas as pd
i = 0
urls = pd.read_csv("urls.csv")
urls = urls.urls.values
def clean(text):
    text = re.sub('<.*>','',text)
    text = re.sub('\n+','',text)
    text = re.sub(' +',' ',text)
    return text.strip()
data = []  

for url in urls:
    print(i,">>",url)
    # try:
    s = requests.Session()
    s.mount(url, HTTPAdapter(max_retries=3))
    page = requests.get(url, allow_redirects=True)
    print(page.status_code)
    
    soup = BeautifulSoup(page.content, "lxml")
    title = str(soup.title)
    title = clean(title)

    # print(dir(soup))
    text = str(soup.text)
    if len(text)<100:

        data.append({"n":i,"url":url,"title":None,"text":None})
    else:    
        text = [i for i in np.unique(sent_tokenize(text)) if len(i)<1000]
        text = [clean(i) for i in text]
        data.append({"n":i,"url":url,"title":title,"text":text})
    print("====================")
    i+=1
    # break

In [317]:
import pandas as pd
import urllib
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests
from requests.adapters import HTTPAdapter
from fake_useragent import UserAgent
from requests.exceptions import ConnectionError
import re
import json
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize 
i = 0
urls = pd.read_csv("Untitled Folder/good_urls.csv")
urls = urls.urls.values
def clean(text):
    text = re.sub('<.*>','',text)
    text = re.sub('\n+','',text)
    text = re.sub(' +',' ',text)
    return text.strip()
data = []  
# titles = []
# texts = []
for url in urls:
    print(i,">>",url)
    # try:
    s = requests.Session()
    s.mount(url, HTTPAdapter(max_retries=3))
    page = requests.get(url, allow_redirects=True)
    # if page.status_code == 404:
        
    print(page.status_code)
    
    soup = BeautifulSoup(page.content, "lxml")
    title = str(soup.title)
    title = clean(title)

    # print(dir(soup))
    text = str(soup.text)
    if len(text)<100:

        data.append({"n":i,"url":url,"title":None,"text":None})
    else:    
        text = [i for i in np.unique(sent_tokenize(text)) if len(i)<1000]
        text = [clean(i) for i in text]
        data.append({"n":i,"url":url,"title":title,"text":text})
    print("====================")
    i+=1
    break

0 >> https://www.factorybuys.com.au/products/euro-top-mattress-king
200


I used openai to generate samples (about 120)

In [ ]:
from openai import OpenAI

client = OpenAI()

processed_data = []
for d in data_titled:
    print(d['n'])
    title = d['title']
    body = " ".join(" ".join(d['text']).split()[0:400])

    response = client.chat.completions.create(
      model="gpt-3.5-turbo-1106",
      response_format={ "type": "json_object" },
      messages=[
        {"role": "system", "content": f""""given a title: {title}\nand product description:{body}\n\nReturn json product name, product type, product color, list of 3 attributes.\nIf product is not a furnitue, return None.\nIf description contains multiple products return None"""},
      ],
        max_tokens = 100,
        temperature = 0,
        n = 1
    )
    print(response)
    d['text_short'] = body
    d['processed'] = response.choices[0].message.content

In [ ]:
data_titled[0]['processed']
new_data = []
for d in data_titled:
    new_data.append(f"""[INST]{d['text_short']}\nExtract product name, product type, product color, list of 3 attributes. [/INST]\n{d['processed']}""")
pd.DataFrame(zip([d['text_short'] for d in data_titled], [d['title'] for d in data_titled], [d['processed'] for d in data_titled], new_data),
             columns=[['text_short','title', 'processed','text']]).to_csv('llama2_dataset2.csv',index=False)


at some point I picked only these samples that returned title during the parsing, so I could use it for validation, but had no time for testing. 

In [259]:
model_name = "NousResearch/Llama-2-7b-chat-hf"

dataset_name = "llama2_dataset2"

new_model = "llama-2-7b-product-parse2"

lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
# device_map = {"": 0}
device_map = 'auto'

In [261]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    cache_dir="./data/models/"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    # dataset_text_field="conversation",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


In [10]:
# Train model
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ubuntu/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,2.273900
50,1.909100


TrainOutput(global_step=70, training_loss=2.0263529641287668, metrics={'train_runtime': 478.7046, 'train_samples_per_second': 0.581, 'train_steps_per_second': 0.146, 'total_flos': 6942979218407424.0, 'train_loss': 2.0263529641287668, 'epoch': 2.0})

In [13]:

del trainer
import gc
gc.collect()
gc.collect()

0

In [325]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    # quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map=device_map,
    cache_dir="./data/models/"
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [45]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000, return_full_text=False, repetition_penalty=1.1)


In [ ]:
f = open('./Untitled Folder/untitled&titled_data.json')
dtst = json.load(f)
for d in tqdm(dtst):
    try:
        prompt = f"""[INST]{d['text_short']}\nExtract product name, product type, product color, list of 3 attributes. [/INST]\n"""
        result = pipe(prompt)
        print(d['n'], result[0]['generated_text'])
        d['result'] = result[0]['generated_text']
    except Exception as e:
        print("Error OTHER: ", e)
        d['result'] = None
    # break

  0%|          | 1/231 [00:03<12:38,  3.30s/it]

1 {
  "product_name": "Euro Top Mattress",
  "product_type": "Furniture",
  "product_color": "Medium Thickness: 32cm",
  "attributes": ["Plush Euro Top padding", "5 zoned pocket springs", "Independent coil system"]
}


  1%|          | 2/231 [00:05<11:09,  2.92s/it]

2 {
  "product_name": "Beadlight Cirrus LED Reading Light",
  "product_type": "Lighting",
  "product_color": "Black",
  "attributes": ["Energy efficient", "Handmade", "Private jet aircraft inspired"]
}


  1%|▏         | 3/231 [00:09<12:17,  3.24s/it]

...

533 {
  "product_name": "Grey Wool Rug",
  "product_type": "Rug",
  "product_color": "Grey",
  "attributes": ["Wool", "Handmade", "Scandinavian Design"]
}


 80%|████████  | 185/231 [09:24<02:06,  2.75s/it]

536 {
  "product_name": "Modern Theatre Lounge",
  "product_type": "Furniture",
  "product_color": "Black",
  "attributes": ["Made to order", "Solid timber frame", "No sag springs"]
}


 81%|████████  | 186/231 [09:27<02:04,  2.77s/it]

539 {
  "product_name": "Satin Napkins",
  "product_type": "Party Supplies",
  "product_color": "N/A",
  "attributes": ["18 x 18 Inch", "Satin", "Rhinestone"]
}


this would be good score, if it was tested on representative set of evaluation data, but there was not enough data.

In [344]:

from evaluate import load
import evaluate
rouge = evaluate.load('rouge')
validate_res = []
for d in dat:
    if 'processed' in d.keys():

        validate_res.append(rouge.compute(predictions=[d['result']], references=[d['processed']]))
    # break

np.mean([v['rougeLsum'] for v in validate_res])

0.5647831741610471

Method for extraction, task solution

In [ ]:
class Extractor:
def __init__():
    new_model = "llama-2-7b-product-parse2"
    base_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        low_cpu_mem_usage=True,
        return_dict=True,
        # quantization_config=bnb_config,
        torch_dtype=torch.float16,
        device_map='auto',
        cache_dir="./data/models/"
    )
    self.model = PeftModel.from_pretrained(base_model, new_model)
    self.model = model.merge_and_unload()
    
    # Reload tokenizer to save it
    self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    self.tokenizer.pad_token = tokenizer.eos_token
    self.tokenizer.padding_side = "right"

    self.pipe = pipeline(task="text-generation", model=self.model, tokenizer=self.tokenizer, max_length=1000, return_full_text=False, repetition_penalty=1.1)

def clean(self, text: str) -> str:
    text = re.sub('<.*>','',text)
    text = re.sub('\n+','',text)
    text = re.sub(' +',' ',text)
    return text.strip()
    
def parse_url(self, url: str) -> Dict[Any, Any]:#this is a bit of a pseudocode :)
    print(i,">>",url)
    # try:
    s = requests.Session()
    s.mount(url, HTTPAdapter(max_retries=3))
    page = requests.get(url, allow_redirects=True)
    # if page.status_code == 404:
        
    print(page.status_code)
    
    soup = BeautifulSoup(page.content, "lxml")
    title = str(soup.title)
    title = clean(title)

    # print(dir(soup))
    text = str(soup.text)
    if len(text)<100:

        return None #probably domain for sale
    else:    
        text = [i for i in np.unique(sent_tokenize(text)) if len(i)<1000]
        text = [clean(i) for i in text]
        return {"n":i,"url":url,"title":title,"text":text}
        
def extract(self, url: str) -> Dict[Any, Any]:
    data= self.parse_url(url)

    prompt = f"""[INST]{d['text_short']}\nExtract product name, product type, product color, list of 3 attributes. [/INST]\n"""
    result = self.pipe(prompt)
    data['result'] = result[0]['generated_text']
    return data

In [157]:
# with open('final_res.json', 'w') as f:
#     json.dump(dtst, f)

In [160]:

for d in dtst:
    try:
        d['res_fixed'] = json.loads(d['result'])
    except Exception as e:
        # print(d['n'],d['result'])
        multiple = re.sub('}\n?{','}~{',re.sub('\n','',d['result'])).split('~')
        multiple = [m for m in multiple if re.findall(r'^ ?\{.*\} ?$',m)]
        try:
            multiple = [json.loads(m) for m in multiple]
            d['res_fixed'] = multiple
        except Exception as e:
            d['res_fixed'] = None
        # print("===")

In [187]:
results_ok = [d['res_fixed'] for d in dtst if d['res_fixed'] != None]

would be a good idea to filter out non furniture objects during parsing, or use classifier (instead of zero shot) to do cleaning, but I had no time

In [302]:
# pipe = pipeline(model="facebook/bart-large-mnli")
res_analysis = []
for res in results_ok:
    if isinstance(res, dict):
        res_analysis.append(res)
        # check_res = res
        # label = pipe(check_res['product_type'], candidate_labels=["furniture", "not furniture"])
        # print(label['labels'][0],check_res['product_type'])
    elif isinstance(res, list) and len(res)>=1:
        for r in res:
            if isinstance(r, dict):
                res_analysis.append(r)
                # check_res = r
                # label = pipe(check_res['product_type'], candidate_labels=["furniture", "not furniture"])
                # print(label['labels'][0],check_res['product_type'])


In [205]:
names = [r['product_name'] for r in res_analysis]
types = [r['product_type'] for r in res_analysis]
colors = [r['product_color'] for r in res_analysis]
attributes = [r['attributes'] for r in res_analysis]

In [81]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [272]:
def plot_vals(vals, name, thr = 1):
    vals =pd.DataFrame(vals)[0].value_counts()
    vals = dict(vals[vals>thr])
    output_notebook()
    fruits = list(vals.keys())
    counts = list(vals.values())
    
    p = figure(x_range=fruits, height=350, width=1900, title=name,
               toolbar_location=None, tools="")
    
    p.vbar(x=fruits, top=counts, width=0.9)
    
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    
    show(p)

Should have made the dataset better to specify, what type of furniture it is, if not assign other label to

In [273]:
plot_vals(types, 'Product types')
plot_vals(colors, 'colors')
plot_vals(attributes, 'attributes')

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [38]:
import torch  
from transformers import BertTokenizer,BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') 
model = BertModel.from_pretrained("bert-base-uncased")

In [16]:
encoded_input = tokenizer(names, return_tensors='tf',padding=True, truncation=True)
embeddings = model(encoded_input)

In [53]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/LaBSE', device ="cpu")

In [54]:
with torch.no_grad():
    embeddings = model.encode(names)


In [82]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scalar = StandardScaler() 
scaled = scalar.fit_transform(embeddings)

pca = PCA(n_components=2)
pca.fit(scaled)
emb_2d = pca.transform(scaled)


In [147]:

from bokeh.plotting import figure, output_file, show 
from bokeh.models import HoverTool 
import pandas as pd 
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Viridis256
# create some data 
data = pd.DataFrame({'x':  emb_2d[:,0], 
                     'y': emb_2d[:,1], 
                     'labels': names,
                     'types': types}) 

source = bpl.ColumnDataSource(data)
import random

get_index = random.randrange(len(Viridis256))

palette = random.sample(Viridis256, len(data['types'].unique()))

color_map = bmo.CategoricalColorMapper(factors=data['types'].unique(),
                                   palette=palette)
                     
p = figure(title='Names cloud',width=1200, height=1200) 
  
# add scatter plot 

p.scatter(x='x', y='y',  size=10,  color={'field': 'types', 'transform': color_map}, source=source
         ) 
  
# add hover tool 
hover = HoverTool(tooltips=[('', '@labels')]) 
p.add_tools(hover) 
  
# show the plot 
show(p) 


In [231]:
col = [ re.sub(r'a','', c) if type(c)==str else None for c in colors]
att = [ ", ".join(a) for a in attributes]
desc = [", ".join([n,c,a,t]) if c!=None else ", ".join([n,a,t]) for c,a,n,t in zip(col,att,names,types)]

In [224]:
from datasets import Dataset
df = pd.DataFrame(zip(names,types,desc))
dataset = Dataset.from_pandas(df)


In [301]:
to_chroma = []
for d in dat:
    try:
        doc = json.loads(d['result'])
        att = ", ".join(doc['attributes'])
        to_chroma.append({"data": f"name:{doc['product_name']}, type: {doc['product_type']}, properties {doc['product_color']} {att}", "ids": d['n'], 'metadatas': {'source':d['url']}})
    except:
        pass
    # break

In [303]:
import chromadb
client = chromadb.Client()
collection = client.create_collection("col1")
# client.delete_collection(name="col1")

In [305]:

collection.add(
    documents=[i['data'] for i in to_chroma],
    ids=[str(i['ids']) for i in to_chroma],
    metadatas = [i['metadatas'] for i in to_chroma]
)

In [306]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
   low_cpu_mem_usage=True,
    return_dict=True,
    device_map='auto',
    cache_dir="./data/models/")
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000, return_full_text=False, repetition_penalty=1.1)


In [271]:
def search(query):
    prompt = f"""[INST]From the following user statement extract object of user's interest Reply with a single word:\n{query}. [/INST]\n"""
    result = pipe(prompt)
    results = collection.query(
        query_texts=[result[0]['generated_text']],
        n_results=3
    )
    return results

In [307]:
search("I want a white chair")

{'ids': [['588', '262', '188']],
 'distances': [[1.0380353927612305, 1.0595630407333374, 1.0695416927337646]],
 'metadatas': [[{'source': 'https://donar.si/products/collodi'},
   {'source': 'https://www.couchpotatocompany.com/collections/best-sellers/products/ro-armchair'},
   {'source': 'https://www.aarniooriginals.com/collections/shop/products/pony'}]],
 'embeddings': None,
 'documents': [['name:Nesting Chair, type: Furniture, properties Wooden frame with industrial waste shell Environmentally friendly, Comfortable, Emotive',
   'name:Ro Armchair, type: Furniture, properties Natural Elegant design, High back, Soft upholstery',
   'name:Pony Chair, type: Furniture, properties Green and White Ergonomic, Playful, Humorous']],
 'uris': None,
 'data': None}

In [318]:
search("I want a dream sofa")

{'ids': [['124', '510', '491']],
 'distances': [[0.7196788191795349, 0.7713944911956787, 0.7796918153762817]],
 'metadatas': [[{'source': 'https://www.thefurnituremegastore.co.uk/products/noir-80x180-cm-wall-mirror'},
   {'source': 'https://www.classic2modern.com/products/100-sofa'},
   {'source': 'https://whataroom.com/collections/new-arrival-rugs/products/arsene-r3475-aqu000'}]],
 'embeddings': None,
 'documents': [['name:Quirky Home Accessories and Furniture, type: Furniture, properties N/A Fabric Sofas, Leather Sofas, Recliner Sofas',
   'name:100 Classic Leather Sofa, type: Furniture, properties Natural Timeless beauty, Inviting appeal, Comfort',
   'name:Customizable Sofa, type: Furniture, properties Various Customizable, Leather or Fabric, Sofa']],
 'uris': None,
 'data': None}

In [321]:
search("can you suggest me a chair for niece who likes horses?")

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'ids': [['188', '691', '600']],
 'distances': [[1.2606855630874634, 1.4421956539154053, 1.451244592666626]],
 'metadatas': [[{'source': 'https://www.aarniooriginals.com/collections/shop/products/pony'},
   {'source': 'https://www.collectioni.com/products/noahs-ark-lounge-swivel-chair'},
   {'source': 'https://www.angliarecliners.co.uk/products/swan/'}]],
 'embeddings': None,
 'documents': [['name:Pony Chair, type: Furniture, properties Green and White Ergonomic, Playful, Humorous',
   'name:Sheepskin Stool, type: Furniture, properties Natural Limited edition, Made from sheepskin, Handcrafted',
   'name:Swan Sofa, type: Furniture, properties Black Adjustable headrest, Individually adjustable headrests, Available in 5 different grades of leather']],
 'uris': None,
 'data': None}

In [323]:
search("give me a big wardrobe")

{'ids': [['213', '201', '124']],
 'distances': [[1.0120916366577148, 1.1255121231079102, 1.145043969154358]],
 'metadatas': [[{'source': 'https://furnitica-vinova.myshopify.com/products/enim-donec-pede'},
   {'source': 'https://www.castlery.com/products/spot-shelf'},
   {'source': 'https://www.thefurnituremegastore.co.uk/products/noir-80x180-cm-wall-mirror'}]],
 'embeddings': None,
 'documents': [['name:Dress, type: Clothing, properties Black Fast Search, List of 3 Attributes',
   'name:Modern Furniture, type: Furniture, properties Multiple Colors Online Shopping, Castlery, Modern Design',
   'name:Quirky Home Accessories and Furniture, type: Furniture, properties N/A Fabric Sofas, Leather Sofas, Recliner Sofas']],
 'uris': None,
 'data': None}